# Predicting the Sale Price of Bulldozers

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Problem Definition

> How well can we predict the future sale price of a bulldozer, given its characteristics and previous examples of similar bulldozers prices?

The evaluation metric for this competition is the RMSLE (root mean squared log error) between the actual and predicted auction prices.

The goal is therefore to minimize this error

# Data

The data is from the Kaggle Bluebook for Bulldozers competition: https://www.kaggle.com/c/bluebook-for-bulldozers/data

The link contains everything necessary: the train, test, validation data as well as the feature dictionary